## Fake News Detection In Dravidian Languages

In [73]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import fasttext
import tokenizers

In [74]:
data = pd.read_csv("labelled_train_set.csv")

In [75]:
main_test = pd.read_csv('unlabelled_test1.csv')

In [76]:
data.Type = data.Type.str.replace(" ","_")

In [77]:
# data["News/Comment"]=data["News/Comment"].str.replace("\u200d",'').replace("\n","")

In [78]:
data.Type.value_counts()

Type
FALSE           1130
MOSTLY_FALSE     230
HALF_TRUE        138
PARTLY_FALSE      38
MOSTLY_TRUE        1
Name: count, dtype: int64

In [79]:
data["Type"] = "__label__" + data["Type"].astype('str')

In [80]:
data['description'] = data["Type"] + " " + data["News/Comment"]

In [81]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(data, test_size = 0.2)

In [82]:
train.iloc[15,1]

'പരാന്നഭോജി എന്ന വാക്കിനെ ഗൂഗിൾ നിർവചിച്ചിരിക്കുന്നത് "മുതലാളിത്തം" എന്നാണ്.'

In [83]:
train.to_csv("dra.train", columns = ["description"], index = False , header = False)
test.to_csv("dra.test", columns = ["description"], index = False , header = False)

In [84]:
input_path_train = "dra.train"
input_path_test = "dra.test"

In [85]:
def print_results(model, input_path, k):
    num_records, precision_at_k, recall_at_k = model.test(input_path, k)
    f1_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)

    print("records\t{}".format(num_records))
    print("Precision@{}\t{:.3f}".format(k, precision_at_k))
    print("Recall@{}\t{:.3f}".format(k, recall_at_k))
    print("F1@{}\t{:.3f}".format(k, f1_at_k))
    print()

### Normal Model 

In [86]:
import fasttext 
model = fasttext.train_supervised(input=input_path_train,lr=0.1,lrUpdateRate=1000,wordNgrams=3,epoch=50,loss='ova') 

Read 0M words
Number of words:  7641
Number of labels: 5
Progress: 100.0% words/sec/thread:  337860 lr:  0.000000 avg.loss:  0.765016 ETA:   0h 0m 0s


In [87]:
model.test("dra.test")

(269, 0.7211895910780669, 0.7211895910780669)

In [88]:
main_test

,ID,News/Comment
0,TEST_1,വിഴിഞ്ഞത്ത് തീരദേശവാസികള്‍ ആക്രമിച്ചപ്പോള്‍ മു...
1,TEST_2,കുരിശിന് മുന്നില്‍ കൈകൂപ്പി നില്‍ക്കുന്ന പിണറാ...
2,TEST_3,1971ലെ തലശ്ശേരി കലാപത്തില്‍ പങ്കുണ്ടായിരുന്ന പ...
3,TEST_4,ഗവര്‍ണര്‍ ആരിഫ് മുഹമ്മദ് ഖാനുമായുള്ളതര്‍ക്കം പ...
4,TEST_5,എല്ലാ മേഖലകളിലും കേരളം തകര്‍ന്നടിഞ്ഞുവെന്ന് സമ...
...,...,...
127,TEST_128,ന്യൂയോർക്ക് ടൈംസിന്റെ എഡിറ്റർ ജോസഫ് ഹോപ്പ് ഒരു...
128,TEST_129,ഷിർദിയിലെ സായി ക്ഷേത്രത്തിൽ ഹിന്ദുക്കൾ നൽകുന്ന...
129,TEST_130,ഉത്തര്പ്രദേശിലെ മദ്രസയില് യുവാവ് കുട്ടിയെ ക്രൂ...
130,TEST_131,ഐഎസ്ആർഒ ചന്ദ്രയാൻ -3 മിഷൻ ശനിയിൽ ഇറങ്ങുന്നതിന്...


In [89]:
final = main_test["News/Comment"].str.replace("\u200d",'').replace("\n","").tolist()

In [90]:
for k in range(1, 3):
    print('train metrics:')
    print_results(model, input_path_train, k)

    print('test metrics:')
    print_results(model, input_path_test, k)

train metrics:
records	1053
Precision@1	0.970
Recall@1	0.970
F1@1	0.970

test metrics:
records	269
Precision@1	0.721
Recall@1	0.721
F1@1	0.721

train metrics:
records	1053
Precision@2	0.486
Recall@2	0.972
F1@2	0.648

test metrics:
records	269
Precision@2	0.450
Recall@2	0.900
F1@2	0.600



In [91]:
df = pd.DataFrame(columns=["ID","Predicted Type"])

In [92]:
df['ID'] = main_test['ID']

In [93]:
l = []
for item in final:
   item = item.replace("\n"," ")
   l.append(model.predict(item)[0][0])

In [94]:
df['Predicted Type'] = l

In [95]:
df['Predicted Type']= df['Predicted Type'].str.replace("__label__","").replace("_","")

In [96]:
name_dict = {'FALSE':0,"HALF_TRUE":1,"PARTLY_FALSE":3}

In [100]:
df['Predicted Type'].tolist()

['FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'HALF_TRUE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'HALF_TRUE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 'FALSE',
 '

In [103]:
df['Predicted Type']=df['Predicted Type'].apply(lambda x: name_dict[x])

In [107]:
df.to_csv("fasttext_dravidian.csv",index=False)